In [25]:
import glob
import os
from collections import Counter
from itertools import combinations, product

import numpy as np
np.random.seed(18012023)

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import seaborn as sb

from sklearn.metrics import pairwise_distances
import lxml.etree
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'Arial'
from IPython.display import display

In [26]:
fig_dir = '../figures'
if not os.path.isdir(fig_dir):
    os.mkdir(fig_dir)

#### Load metadata

In [27]:
meta_df = pd.read_excel('../data/metadata_corrected.xlsx')
meta_df = meta_df[meta_df['exclude'] != 'x']
meta_df.sample(10)

,id,title,author,provenance,date_range,genre,subgenre,exclude
211,spel_van_abraham_en_sara,Spel van Abraham en Sara,NaN,cdrom-mnl,1440-1460,Dramatiek,NaN,NaN
60,vanden_vier_vingheren_ende_vanden_dume,Expositie vanden viere vingheren ende vanden dume,NaN,cdrom-mnl,1400-1420,Epiek,Didactiek,NaN
233,spiegel_historiael__4_velthem__fragm_be,Spiegel historiael (P4),Lodewijk van Velthem,cdrom-mnl,1315-1350,Epiek,Historiografie,NaN
203,sinte_franciscus_leven,Sinte Franciscus leven,Jacob van Maerlant,cdrom-mnl,1300-1350,Epiek,Heiligenleven,NaN
66,florigout_fragm_db,Florigout,NaN,cdrom-mnl,1375-1400,Epiek,Ridder,NaN
177,rijmkroniek_van_holland,Rijmkroniek van Holland,Melis Stoke,cdrom-mnl,1365-1385,Epiek,Historiografie,NaN
107,lantsloot_van_der_haghedochte,Lantsloot van der Haghedochte,NaN,cdrom-mnl,1300-1350,Epiek,Arthur,NaN
55,dystorie_van_saladine_h,Dystorie van Saladine,NaN,cdrom-mnl,1479-1483,Epiek,Kruisvaart,NaN
69,floovent,Flovent,NaN,cdrom-mnl,1440-1460,Epiek,Karel,NaN
43,der_mannen_ende_vrouwen_heimelijcheit,Der mannen ende vrouwen heimelijcheit,NaN,cdrom-mnl,1351-1351,Epiek,Didactiek,NaN


In [28]:
def get_verse_groups(verses, size=2, intertexts=False):
    for i in range(len(verses) - (size - 1)):
        if not intertexts:
            yield ' / '.join(verses[i : i + size])
        else:
            its = Counter(verses[i : i + size])
            if None in its:
                yield None
            elif len(its) > 1:
                yield 'overlap'
            else:
                yield list(its.keys())[0]

In [29]:
def parse_xml(fn, rm_interpol=False):
    try:
        tree = lxml.etree.parse(fn)
    except OSError:
        print(f'- Could not load {fn}')
        return None
        
    if rm_interpol:
        for interpolation in tree.xpath("//interpolation"):
            interpolation.getparent().remove(interpolation)
        
    for line_node in tree.iterfind('//l'):
        try:
            intertext_id = line_node.attrib['intertext']
        except KeyError:
            intertext_id = None
        
        tokens_ = line_node.attrib['tokens'].split()
        lemmas_ = []
        
        lemma_tags = [l.text for l in line_node.iterfind('.//lemma')]
        pos_tags = [p.text for p in line_node.iterfind('.//pos')]
        
        for lemma, pos in zip(lemma_tags, pos_tags):
            for l, p in zip((lemma or '').split('+'), (pos or '').split('+')):
                    if p == 'NOU-P':
                        lemmas_.append('NOU-P')
                    else:
                        lemmas_.append(l)
    
        yield tokens_, lemmas_, intertext_id

In [30]:
GROUP_SIZE = 2

titles, tokens, lemmas, intertexts = [], [], [], []

for title, group in tqdm(meta_df.groupby('title')):
    work_tokens, work_lemmas, work_intertexts = [], [], []
    
    for id_ in sorted(group['id']):
        for tok, lem, intertext_id in parse_xml(f'../data/xml_galahad_BY-split/{id_}.xml'):
            work_tokens.append(tok)
            work_lemmas.append(lem)
            work_intertexts.append(intertext_id)
    
    verse_tokens = [' '.join(v) for v in work_tokens]
    verse_lemmas = [' '.join(v) for v in work_lemmas]

    verse_group_tokens = list(get_verse_groups(verse_tokens, size=GROUP_SIZE))
    verse_group_lemmas = list(get_verse_groups(verse_lemmas, size=GROUP_SIZE))
    verse_group_intertexts = list(get_verse_groups(work_intertexts, size=GROUP_SIZE, intertexts=True))

    tokens.extend(verse_group_tokens)
    lemmas.extend(verse_group_lemmas)
    intertexts.extend(verse_group_intertexts)
    titles.extend([title] * len(verse_group_lemmas))

100%|█████████████████████████████████████████| 207/207 [00:11<00:00, 17.91it/s]


In [31]:
df = pd.DataFrame(zip(titles, tokens, lemmas, intertexts), columns=('title', 'tokens', 'lemmas', 'intertext'))

In [32]:
df

,title,tokens,lemmas,intertext
0,AB recht ende averecht,Aensiet dese vrouwen hoe si gaen / Besiet hoe haer tuten staen,aanzien deze vrouw hoe zij gaan / bezien hoe haar tuut staan,None
1,AB recht ende averecht,Besiet hoe haer tuten staen / Claer dat si hen blanketten,bezien hoe haar tuut staan / klaar dat zij zij blanketten,None
2,AB recht ende averecht,Claer dat si hen blanketten / Die cleeder soe lanc dat si hen letten,klaar dat zij zij blanketten / die kleed zo lang dat zij zij letten,None
3,AB recht ende averecht,Die cleeder soe lanc dat si hen letten / Ende sleypen hen nae al op die eerde,die kleed zo lang dat zij zij letten / en slepen zij na al op die aarde,None
4,AB recht ende averecht,Ende sleypen hen nae al op die eerde / Fi diere vuylder hoverde,en slepen zij na al op die aarde / fi die vuil hovaart,None
...,...,...,...,...
873418,Wrake van Ragisel,Nu selewi swigen van desen / Ende van enen jongelinc vord lesen,nu zullen wij zwijgen van deze / en van een jongeling voort lezen,None
873419,Wrake van Ragisel,Ende van enen jongelinc vord lesen / Die te hove nu sal comen,en van een jongeling voort lezen / die te hof nu zullen komen,None
873420,Wrake van Ragisel,Die te hove nu sal comen / Eest alsict hebbe vernomen,die te hof nu zullen komen / zijn het als zij hebben vernemen,None
873421,Wrake van Ragisel,Eest alsict hebbe vernomen / Soe salmen noch van hem sien,zijn het als zij hebben vernemen / zo zullen men nog van hij zien,None


In [33]:
def tokenizer(text):
    return text.replace(' / ', ' ').lower().strip().split()

def add_rhyme_column(df):
    rhyme_words = []
    for lemmas in df['lemmas']:
        rhymes = []
        for verse in lemmas.split(' / '):
            words = verse.strip().split()
            if words:  # Ensure the verse is not empty
                rhymes.append(words[-1])
            else:
                rhymes.append('')  # Use an empty string or placeholder
        rhyme_words.append(' '.join(rhymes))
    df['rhyme'] = rhyme_words
    return df

In [34]:
df = add_rhyme_column(df)
df.head()

,title,tokens,lemmas,intertext,rhyme
0,AB recht ende averecht,Aensiet dese vrouwen hoe si gaen / Besiet hoe haer tuten staen,aanzien deze vrouw hoe zij gaan / bezien hoe haar tuut staan,None,gaan staan
1,AB recht ende averecht,Besiet hoe haer tuten staen / Claer dat si hen blanketten,bezien hoe haar tuut staan / klaar dat zij zij blanketten,None,staan blanketten
2,AB recht ende averecht,Claer dat si hen blanketten / Die cleeder soe lanc dat si hen letten,klaar dat zij zij blanketten / die kleed zo lang dat zij zij letten,None,blanketten letten
3,AB recht ende averecht,Die cleeder soe lanc dat si hen letten / Ende sleypen hen nae al op die eerde,die kleed zo lang dat zij zij letten / en slepen zij na al op die aarde,None,letten aarde
4,AB recht ende averecht,Ende sleypen hen nae al op die eerde / Fi diere vuylder hoverde,en slepen zij na al op die aarde / fi die vuil hovaart,None,aarde hovaart


In [35]:
optim_vs = 7000
optim_rw = 0.1693877551020408
optim_th = 0.42202047865566794

In [36]:
vec = TfidfVectorizer(max_features=optim_vs, min_df=2,
                      tokenizer=tokenizer, token_pattern=None).fit(df['lemmas'])

## Nearest neighbors

In [37]:
def nearest_verses(title1, title2, base_df, vectorizer, prop_filt=2, rhyme_weight=.15):
    A = base_df[base_df['title'] == title1]
    B = base_df[base_df['title'] == title2]
    
    AX = vec.transform(A['lemmas']) + rhyme_weight * vec.transform(A['rhyme'])
    BX = vec.transform(B['lemmas']) + rhyme_weight * vec.transform(B['rhyme'])
    
    nearest = []
    for i, ax in enumerate(AX):
        distances = pairwise_distances(ax, BX, metric='cosine').flatten()
        top_i = distances.argsort()[0]
        nearest.append((A.iloc[i]['tokens'], B.iloc[top_i]['tokens'],
                        A.iloc[i]['lemmas'], B.iloc[top_i]['lemmas'],
                        A.iloc[i]['title'], B.iloc[top_i]['title'],
                        A.iloc[i]['intertext'], B.iloc[top_i]['intertext'],
                        distances[top_i]))

    nearest = pd.DataFrame(nearest,
                           columns=['tokens1', 'tokens2', 'lemmas1', 'lemmas2',
                                    'title1', 'title2', 'intertext1', 'intertext2',
                                    'distance'])

    nearest = nearest[nearest['title1'] != nearest['title2']]
    nearest = nearest[
                      (nearest['lemmas1'].str.count('NOU-P') < prop_filt) & \
                      (nearest['lemmas1'].str.count('NOU-P') < prop_filt) 
                     ]
    return nearest

In [38]:
nearest_df = nearest_verses('Der leken spieghel', 'Jans teesteye',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/lekenspiegel-teesteye.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
4708,Ende oec niet stelen wats ghesciet / Nieman doeden noch verslaen,Ende oec niet stelen wats ghesciet / Nieman doden noch verslaen,en ook niet stelen wat geschieden / niemand doden noch verslaan,en ook niet stelen wat geschieden / niemand doden noch verslaan,Der leken spieghel,Jans teesteye,None,None,0.000000e+00
4709,Nieman doeden noch verslaen / Gheen valsch orconde doen verstaen,Nieman doden noch verslaen / Gheen valsche orconde doen verstaen,niemand doden noch verslaan / geen vals oorkonde doen verstaan,niemand doden noch verslaan / geen vals oorkonde doen verstaan,Der leken spieghel,Jans teesteye,None,None,0.000000e+00
4699,Elc mensche die sal / Gode minnen bouen al,Elc mensche die sal / Gode minnen boven al,elk mens die zullen / NOU-P minnen boven al,elk mens die zullen / NOU-P minnen boven al,Der leken spieghel,Jans teesteye,None,None,1.110223e-16
3501,Sal men gaen daer toe / Wijslijc ende met maten,Salmen gaen daer toe / Wijsselijc ende met maten,zullen men gaan daar toe / wijselijk en met maat,zullen men gaan daar toe / wijselijk en met maat,Der leken spieghel,Jans teesteye,None,None,1.110223e-16
2732,Want soe die nideghe langher leeft / Soe hi meer onwils heeft,Want so die nideghe langher leeft / So hi meer sijns onwillen heeft,want zo die nijdige lang leven / zo hij meer onwil hebben,want zo die nijdige lang leven / zo hij meer zijn onwil hebben,Der leken spieghel,Jans teesteye,None,None,4.706122e-03
11201,Daer hi elken loen sal gheuen / Van dat hi hier heeft bedreuen,Hi elken loen sal gheven / Van dat hi hier heeft bedreven,daar hij elk loen zullen geven / van dat hij hier hebben bedrijven,hij elk loen zullen geven / van dat hij hier hebben bedrijven,Der leken spieghel,Jans teesteye,None,None,1.312535e-02
14677,Ende elken loen zal gheuen / Van dat hi hier heeft bedreuen,Hi elken loen sal gheven / Van dat hi hier heeft bedreven,en elk loen zullen geven / van dat hij hier hebben bedrijven,hij elk loen zullen geven / van dat hij hier hebben bedrijven,Der leken spieghel,Jans teesteye,None,None,1.391072e-02
4707,Anders mans wijf onderwinden niet / Ende oec niet stelen wats ghesciet,Anderre wive onderwenden niet / Ende oec niet stelen wats ghesciet,ander man wijf onderwinden niet / en ook niet stelen wat geschieden,ander wijf onderwinden niet / en ook niet stelen wat geschieden,Der leken spieghel,Jans teesteye,None,None,2.557167e-02
9378,Maer anebeedden een calf / Datsi van ere ghieten daeden,Maer si aenbeedden een calf / Dat si goten van ere,maar aanbeden een kalf / dat zij van een gieten doen,maar zij aanbeden een kalf / dat zij gieten van een,Der leken spieghel,Jans teesteye,None,None,3.199644e-02
5333,Ende elken loen gheuen / Van dat hi hier heeft bedreuen,Hi elken loen sal gheven / Van dat hi hier heeft bedreven,en elk loen geven / van dat hij hier hebben bedrijven,hij elk loen zullen geven / van dat hij hier hebben bedrijven,Der leken spieghel,Jans teesteye,None,None,3.232752e-02


In [39]:
nearest_df = nearest_verses('Melibeus', 'Dietsche doctrinale',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/melibeus-dietschedoctrinale.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
3659,Seneca seghet noch dit woert / Dat nieman bat toe en hoert,Noch seit seneca dit woert / Dat niemanne bat toe en hoert,NOU-P zeggen nog dit woord / dat niemand bet toezenden ne hoeren,nog zeggen NOU-P dit woord / dat niemand bet toezenden ne hoeren,Melibeus,Dietsche doctrinale,None,None,1.110223e-16
3245,Gheeft mi op die wrake / Ic sal lonen die sake,Hi sprect gheeft mi op die wrake / Ende ic sal lonen die sake,geven ik op die wrake / ik zullen lonen die sake,hij spreken geven ik op die wrake / en ik zullen lonen die sake,Melibeus,Dietsche doctrinale,None,None,4.039992e-02
1884,Die van vele lieden ontsien es / Moet vele lieden ontsien weder,Vele lieden hi moet van dien / Vele lieden weder ontsien,die van veel lieden ontzien zijn / moeten veel lieden ontzien weder,veel lieden hij moeten van dat / veel lieden weder ontzien,Melibeus,Dietsche doctrinale,None,None,4.924360e-02
2341,Ende so wien men ontsiet / Ende mach ghemint wesen niet,Soe wien datmen ontsiet / En mach ghemint wesen niet,en zo wie men ontzien / en mogen minnen wezen niet,zo wie dat men ontzien / ne mogen minnen wezen niet,Melibeus,Dietsche doctrinale,None,None,6.233281e-02
1267,Haddic enen voete inden grave / Nochtan so woudic leren,Haddic enen voet int graf / Nochtan soudic willen leren,hebben ik een voet in de graf / nochtan zo willen ik leren,hebben ik een voet in het graf / nochtan zullen ik willen leren,Melibeus,Dietsche doctrinale,None,None,6.543936e-02
1352,Die wise paeus Innocentius / Scrijft in sine boeke aldus,Die paus innocentius / Scrijft in sinen boeke aldus,die wijs paus NOU-P / schrijven in zijn beuk aldus,die paus NOU-P / schrijven in zijn beuk aldus,Melibeus,Dietsche doctrinale,None,None,7.656346e-02
1026,Salomon seghet die hoedt sinen mont / Hoedt sijn ziele talre stont,Soe wie hoeden can sinen mont / Hoet sine ziele talre stont,NOU-P zeggen die hoeden zijn mond / hoeden zijn ziel te al stonde,zo wie hoeden kunnen zijn mond / hoeden zijn ziel te al stonde,Melibeus,Dietsche doctrinale,None,None,9.188778e-02
2265,Wat si u segghen ende tonen / Hoedt u altoes jeghene honen,Watsi segghen ofte tonen / Hoedt u altoes jeghen honen,wat zij u zeggen en tonen / hoeden u altoos jegen hoon,watsie zeggen ofte tonen / hoeden u altoos jegen hoon,Melibeus,Dietsche doctrinale,None,None,1.077470e-01
1466,Want stercmoedecheyt wet wale / Es een der doghet cardinale,Want hets ene doeght wet dat wale / Van iiij doeghden cardinale,want sterkmoedigheid wetten wel / zijn een de deugd kardinaal,want het zijn een deugd wetten dat wel / van vier deugd kardinaal,Melibeus,Dietsche doctrinale,None,None,1.101937e-01
3215,Roepen ghemeynlijc over al / Datmen cracht met crachten weren sal,Roepen alle al ouer al / Datmen cracht met crachte weren sal,roepen gemeenlijk over al / dat men kracht met kracht weren zullen,roepen al al over al / dat men kracht met kracht weren zullen,Melibeus,Dietsche doctrinale,None,None,1.128608e-01


In [40]:
nearest_df = nearest_verses('Brabantsche yeesten (B5)', 'Dietsche doctrinale',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/BY5-DD.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
2334,Nochtan in allen manieren / Soe was hi hem goedertieren,Ontfermhertech ende goedertieren / Es hi in allen manieren,nochtan in al manier / zo zijn hij hij goedertieren,ontfermhartig en goedertieren / zijn hij in al manier,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.122795
844,Daer hi mochte vroech ende spade / Beide met rade ende met dade,Den bedroefden vroech ende spade / Troesten met rade ende met dade,daar hij mogen vroeg en spade / beide met raad en met daad,de bedroefde vroeg en spade / troosten met raad en met daad,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.150918
1050,Oec doetmen met wisen rade / Datmen met crachten niet en dade,Want men doet met wisen rade / Datmen met crachte niet en dade,ook doen men met wijs raad / dat men met kracht niet ne doen,want men doen met wijze raad / dat men met kracht niet ne doen,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.178780
3921,Souden al te nieute keren / Ende dat hi met Gode ende met eren,Ende hem te sulker neren keren / Die met gode si ende met eren,zullen al te niet keren / en dat hij met NOU-P en met eer,en hij te zulk neer keren / die met NOU-P zijn en met eer,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.191295
1144,Ende dat hi om sinen wille allene / Ende om ander dinc en ghene,Ende oec om ander dinc en ghene / Dan om dese viere allene,en dat hij om zijn wil alleen / en om ander ding ne gene,en ook om ander ding ne gene / dan om deze vuur alleen,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.240038
2983,Ende gherne sach si vrede / Eendrachticheit te waren,Van starcmoedecheiden / Van vreden ende eendrachtecheiden,en gaarne zien zij vrede / eendrachtigheid te zijn,van sterkmoedigheid / van vrede en eendrachtigheid,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.242075
1114,Een grooten pot van metale / Diene daer liet weet men wale,Nemt enen pot van motale / Ende enen anderen daer bi,een groot pot van metaal / die hij daar laten weten men wel,nemen een pot van metaal / en een ander daar bij,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.252661
3432,Ende oec swegher die ghelike / Edewaerts van Enghelant,En mach niet sijn haer ghelike / Ihesus sydrac seit des ghelike,en ook zweger die gelijke / edewaarts van NOU-P,ne mogen niet zijn haar gelijke / NOU-P NOU-P zeggen de gelijke,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.262873
1113,Wart daer vonden heb ic vernomen / Een grooten pot van metale,Nemt enen pot van motale / Ende enen anderen daer bi,worden daar vinden hebben ik vernemen / een groot pot van metaal,nemen een pot van metaal / en een ander daar bij,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.268585
3188,Van rechte den Almaenschen rike / Dat si daden groot onghelike,Metten hoeuerdeghen riken / Want si sere ongheliken,van rechte de Almaans rijk / dat zij doen groot ongelijk,met de hoevaardige rijk / want zij zeer ongelijk,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.275432
